In [1]:
import torch
from sparse_ae import *
#import matplotlib.pyplot as plt
from scipy.misc import imresize
from torch.autograd import Variable
import sys
import numpy as np
%matplotlib inline
sys.path.append('../Localization/')
from torchvision import datasets,models,transforms
import os
#from image import readImage
from utils import *

In [2]:
img_path = ['/data/gabriel/LVseg/dataset_img/patches/' ]
#epoch_sae = [2000,5000
#for e_sae in epoch_sae:
epoch_sae = [5000]

for e_sae in epoch_sae:
    for img_path2 in img_path:
        total_im = sum([1 for i in os.listdir(img_path2) if '.png' in i])
    
        n = img_path2[::-1][:img_path2[::-1].find('/')][::-1]
    
        a = SAE(n_in=11,n_h=100,n_out=11,img_path=img_path2,b_size = total_im,patch_size=11,lr = 0.001,gpu=2)
        w,b = train_sae(model=a,epochs = e_sae)
        with open('/data/gabriel/LVseg/'+str(e_sae)+'_'+str(n)+'.p','wb') as f:
            pickle.dump([w,b],f)



0


RuntimeError: cublas runtime error : library not initialized at /b/wheel/pytorch-src/torch/lib/THC/THCGeneral.c:394

In [ ]:
#img_path = '/data/gabriel/LVseg/dataset_img/img/Training'

#img_path = '/data/gabriel/LVseg/dataset_img/img/Training'

pre_train_sae = [True,False]
pre_train_cl = [True,False]
epoch_cl = [5000,10000]
#epoch_cl = [3,1,2]

cl_lr = [0.0001,0.00005]
e_sae = 5000
#num_imgs = 

for cl_lrs in cl_lr:
    for e_cl in epoch_cl:
        
            for p_cl in pre_train_cl:
                nn_l = localnet(img_path='/data/gabriel/LVseg/dataset_img/split/pos/img',
                                label_path='/data/gabriel/LVseg/dataset_img/split/pos/label',b_size=1018,
                                test_fraction=0.1,lr = cl_lrs,gpu=1)

                

                with open('/data/gabriel/LVseg/5000_Training.p') as ff:
                    [w,b] = pickle.load(ff)
                for p in nn_l.modules():
                    if isinstance(p,nn.Conv2d):
                        #print(p.weight.data)
                        #print(p.bias.data)
                        p.weight.data, p.bias.data= w.view(100,1,11,11),b
                        break
                save_dir_name = '/data/gabriel/LVseg/'+str(p_cl)+'_'+str(e_sae)+'_'+str(e_cl)+'_'+str(cl_lrs)
                save_dir_name2 = '/data/gabriel/LVseg/'+str(p_cl)+'_'+str(e_sae)+'_'+str(e_cl)+'_'+str(cl_lrs)+'_best_loss'

                model_name = str(p_cl)+'_'+str(e_sae)+'_'+str(e_cl)+'_'+str(cl_lrs)+'.pth.tar'
                try:
                    shutil.rmtree(save_dir_name)
                    save_dir_name(save_dir_name2)
                    os.makedirs(save_dir_name)
                    os.makedirs(save_dir_name2)
                
                except:
                    os.makedirs(save_dir_name)
                    os.makedirs(save_dir_name2)
                    
                train_lnet(model=nn_l,epochs=e_cl,fname = save_dir_name+'/'+model_name,pretrain=p_cl,save_dir=save_dir_name)

                test_lnet(model=nn_l,fname=save_dir_name+'/'+model_name,save_dir=save_dir_name)
                test_lnet(model=nn_l,fname=save_dir_name+'/'+model_name[:model_name.find('.pth.')]+'_best_loss.pth.tar',save_dir=save_dir_name2)
